[View in Colaboratory](https://colab.research.google.com/github/VictorMarchenkov/res_net/blob/master/DnCNN_keras.ipynb)

По мативам DnCNN-keras-master с GitHub
Пробую подготовит перенос на Colab ...


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL 
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls drive

In [0]:
#data.py
"""
Необходимо запускать для поготовки данных
"""
# -*- coding: utf-8 -*-

import glob
import os
import cv2
import numpy as np
from multiprocessing import Pool

patch_size, stride = 40, 10
aug_times = 1

def data_aug(img, mode=0):
    
    if mode == 0:
        return img
    elif mode == 1:
        return np.flipud(img)
    elif mode == 2:
        return np.rot90(img)
    elif mode == 3:
        return np.flipud(np.rot90(img))
    elif mode == 4:
        return np.rot90(img, k=2)
    elif mode == 5:
        return np.flipud(np.rot90(img, k=2))
    elif mode == 6:
        return np.rot90(img, k=3)
    elif mode == 7:
        return np.flipud(np.rot90(img, k=3))
    
def gen_patches(file_name):

    # read image
    img = cv2.imread(file_name, 0)  # gray scale
    h, w = img.shape
    scales = [1, 0.9, 0.8, 0.7]
    patches = []

    for s in scales:
        h_scaled, w_scaled = int(h*s),int(w*s)
        img_scaled = cv2.resize(img, (h_scaled,w_scaled), interpolation=cv2.INTER_CUBIC)
        # extract patches
        for i in range(0, h_scaled-patch_size+1, stride):
            for j in range(0, w_scaled-patch_size+1, stride):
                x = img_scaled[i:i+patch_size, j:j+patch_size]
                # data aug
                for k in range(0, aug_times):
                    x_aug = data_aug(x, mode=np.random.randint(0,8))
                    patches.append(x_aug)
    
    return patches




In [0]:
!pip -q install keras

In [0]:
# parameters
src_dir = './drive/data/Train400/'
save_dir = './drive/data/npy_data/'
file_list = glob.glob(src_dir+'*.png')  # get name list of all .png files

## num_threads = 16
print('Start...')
# initrialize
res = []
# generate patches
## for i in range(0, len(file_list) ,num_threads):
##     # use multi-process to speed up
##     p = Pool(num_threads)
##     patch = p.map(gen_patches,file_list[i:min(i+num_threads,len(file_list))])
##     #patch = p.map(gen_patches,file_list[i:i+num_threads])
##     for x in patch:
##         res += x
        
##     print('Picture '+str(i)+' to '+str(i+num_threads)+' are finished...')

for i in range(len(file_list)):
    res += gen_patches(file_list[i])
    
# save to .npy
res = np.array(res, dtype='uint8')
print('Shape of result = ' + str(res.shape))
print('Saving data...')
if not os.path.exists(save_dir):
        os.mkdir(save_dir)
np.save(save_dir+'clean_patches.npy', res)
print('Done.')       


In [0]:
#models.py


import numpy as np
from keras.models import *
from keras.layers import Input, Conv2D, BatchNormalization, Activation, Subtract


def DnCNN():
    
    inpt = Input(shape=(None,None,1))
    # 1st layer, Conv+relu
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
    x = Activation('relu')(x)
    # 15 layers, Conv+BN+relu
    for i in range(15):
        x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
        x = Activation('relu')(x)   
    # last layer, Conv
    x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = Subtract()([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model

In [0]:
#main.py

import argparse
import logging
import os, time, glob
import PIL.Image as Image
import numpy as np
import pandas as pd
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from keras.models import load_model
from keras.optimizers import Adam
from skimage.measure import compare_psnr, compare_ssim



## Params
## parser = argparse.ArgumentParser()
## parser.add_argument('--model', default='DnCNN', type=str, help='choose a type of model')
## parser.add_argument('--batch_size', default=128, type=int, help='batch size')
## parser.add_argument('--train_data', default='./drive/data/npy_data/clean_patches.npy', type=str, help='path of train data')
## parser.add_argument('--test_dir', default='./drive/data/Test/Set68', type=str, help='directory of test dataset')
## parser.add_argument('--sigma', default=25, type=int, help='noise level')
## parser.add_argument('--epoch', default=50, type=int, help='number of train epoches')
## parser.add_argument('--lr', default=1e-3, type=float, help='initial learning rate for Adam')
## parser.add_argument('--save_every', default=5, type=int, help='save model at every x epoches')
## parser.add_argument('--pretrain', default=None, type=str, help='path of pre-trained model')
## parser.add_argument('--only_test', default=False, type=bool, help='train and test or only test')
## args = parser.parse_args()

model = 'DnCNN'
batch_size = 128
train_data = './drive/data/npy_data/clean_patches.npy'
test_dir = './drive/data/Test/Set68'
sigma = 25
epoch = 50
lr = 1e-3
save_every = 5
pretrain = None
only_test = False


if not only_test:
    save_dir = './drive/snapshot/save_'+ model + '_' + 'sigma' + str(sigma) + '_' + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) + '/'
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    # log
    logging.basicConfig(level=logging.INFO,format='%(asctime)s %(filename)s[line:%(lineno)d] %(levelname)s %(message)s',
                    datefmt='%Y %H:%M:%S',
                    filename=save_dir+'info.log',
                    filemode='w')
    console = logging.StreamHandler()
    console.setLevel(logging.INFO)
    formatter = logging.Formatter('%(name)-6s: %(levelname)-6s %(message)s')
    console.setFormatter(formatter)
    logging.getLogger('').addHandler(console)
    
##     logging.info(args)
    
else:
    save_dir = '/'.join(pretrain.split('/')[:-1]) + '/'

      
def load_train_data():
    
    logging.info('loading train data...')   
    data = np.load(train_data)
    logging.info('Size of train data: ({}, {}, {})'.format(data.shape[0],data.shape[1],data.shape[2]))
    
    return data

def step_decay(epoch):
    lr = 1e-3
    initial_lr = lr
    if epoch<50:
        lr = initial_lr
    else:
        lr = initial_lr/10
    
    return lr

def train_datagen(y_, batch_size=8):
    
    # y_ is the tensor of clean patches
    indices = list(range(y_.shape[0]))
    while(True):
        np.random.shuffle(indices)    # shuffle
        for i in range(0, len(indices), batch_size):
            ge_batch_y = y_[indices[i:i+batch_size]]
            noise =  np.random.normal(0, sigma/255.0, ge_batch_y.shape)    # noise
            #noise =  K.random_normal(ge_batch_y.shape, mean=0, stddev=args.sigma/255.0)
            ge_batch_x = ge_batch_y + noise  # input image = clean image + noise
            yield ge_batch_x, ge_batch_y
        
def train():
    
    data = load_train_data()
    data = data.reshape((data.shape[0],data.shape[1],data.shape[2],1))
    data = data.astype('float32')/255.0
    # model selection
    if pretrain:
        model_ = load_model(pretrain, compile=False)
    else:   
        if model == 'DnCNN':
            model_ = DnCNN()
    # compile the model
    model_.compile(optimizer=Adam(), loss=['mse'])
    
    # use call back functions
    ckpt = ModelCheckpoint(save_dir+'/model_{epoch:02d}.h5', monitor='val_loss', 
                    verbose=0, period=save_every)
    csv_logger = CSVLogger(save_dir+'/log.csv', append=True, separator=',')
    lr = LearningRateScheduler(step_decay)
    # train 
    history = model_.fit_generator(train_datagen(data, batch_size=batch_size),
                    steps_per_epoch=len(data)//batch_size, epochs=epoch, verbose=1, 
                    callbacks=[ckpt, csv_logger, lr])
    
    return model_

def test(model_):
    
    print('Start to test on {}'.format(test_dir))
    out_dir = save_dir + test_dir.split('/')[-1] + '/'
    if not os.path.exists(out_dir):
            os.mkdir(out_dir)
            
    name = []
    psnr = []
    ssim = []
    file_list = glob.glob('{}/*.png'.format(test_dir))
    for file in file_list:
        # read image
        img_clean = np.array(Image.open(file), dtype='float32') / 255.0
        img_test = img_clean + np.random.normal(0, sigma/255.0, img_clean.shape)
        img_test = img_test.astype('float32')
        # predict
        x_test = img_test.reshape(1, img_test.shape[0], img_test.shape[1], 1) 
        y_predict = model_.predict(x_test)
        # calculate numeric metrics
        img_out = y_predict.reshape(img_clean.shape)
        img_out = np.clip(img_out, 0, 1)
        psnr_noise, psnr_denoised = compare_psnr(img_clean, img_test), compare_psnr(img_clean, img_out)
        ssim_noise, ssim_denoised = compare_ssim(img_clean, img_test), compare_ssim(img_clean, img_out)
        psnr.append(psnr_denoised)
        ssim.append(ssim_denoised)
        # save images
        filename = file.split('/')[-1].split('.')[0]    # get the name of image file
        name.append(filename)
        img_test = Image.fromarray((img_test*255).astype('uint8'))
        img_test.save(out_dir+filename+'_sigma'+'{}_psnr{:.2f}.png'.format(sigma, psnr_noise))
        img_out = Image.fromarray((img_out*255).astype('uint8')) 
        img_out.save(out_dir+filename+'_psnr{:.2f}.png'.format(psnr_denoised))
    
    psnr_avg = sum(psnr)/len(psnr)
    ssim_avg = sum(ssim)/len(ssim)
    name.append('Average')
    psnr.append(psnr_avg)
    ssim.append(ssim_avg)
    print('Average PSNR = {0:.2f}, SSIM = {1:.2f}'.format(psnr_avg, ssim_avg))
    
    pd.DataFrame({'name':np.array(name), 'psnr':np.array(psnr), 'ssim':np.array(ssim)}).to_csv(out_dir+'/metrics.csv', index=True)
    
 
    

model_ = train()
test(model_)       
    


In [0]:
model = load_model(pretrain, compile=False)
test(model)
